In [ ]:
# Deep learning
import torch 
import torchvision
import torch.nn as nn
from torchvision import models
import torchvision.transforms as transforms

# Read Dataset
import cv2
import os
from tqdm import tqdm

# Show Image
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

# Miscelanous
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Dataset Path
TRAIN_PATH = "/kaggle/input/intel-image-classification/seg_train/seg_train"
TEST_PATH = "/kaggle/input/intel-image-classification/seg_test/seg_test"
PRED_PATH = "/kaggle/input/intel-image-classification/seg_pred/seg_pred"

In [ ]:
# device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Datasaet Preparation

In [ ]:
label_mapping = {
    'buildings': 0,
    'forest':1,
    'glacier': 2,
    'mountain': 3,
    'sea': 4,
    'street': 5 
}

In [ ]:
def read_data(path):
    X = []
    y = []
    for category in os.listdir(path):
        category_dir = os.path.join(path, category)
        for img_file in tqdm(os.listdir(category_dir)):
            img_path = os.path.join(category_dir, img_file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (150,150))
            # convert img to torch tensor
            transform = transforms.ToTensor()
            img = transform(img)
            X.append(img)
            y.append(label_mapping[category])
            
    y = torch.tensor(y)
    return (X,y)

In [ ]:
X_train, y_train = read_data(TRAIN_PATH)

## Custom Dataset

In [ ]:
class IntelDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.images = X
        self.labels = y
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        return self.images[idx],self.labels[idx]

In [ ]:
train_dataset = IntelDataset(X_train, y_train)

## Data Loader

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)

## Show Image

In [ ]:
def imshow(axis, inp):
    """Denormalize and show"""
    axis.imshow(inp)

    
def get_label(label_int):
    label_mapping = {
        0: 'buildings',
        1: 'forest',
        2: 'glacier',
        3: 'mountain',
        4: 'sea',
        5: 'street'
    }
    return label_mapping[label_int]

In [ ]:
img, label = next(iter(train_loader))
print(img.size(), label.size())

fig = plt.figure(1, figsize=(16, 4))
grid = ImageGrid(fig, 111, nrows_ncols=(1, 4), axes_pad=0.05)

for i in range(img[:4].size(0)):
    ax = grid[i]
    ax.imshow(img[i].permute(1, 2, 0))
    ax.set_title(get_label(label[i].item()))
    ax.axis('off')

plt.show()

# Modeling

In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Freeze all layers except the classifier
for name, param in model.named_parameters():
    if "fc" not in name: 
        param.requires_grad = False

# add last layer with out_features=num_classes=6
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=6)

model = model.to(device)

In [ ]:
# print(model)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(10):
    model.train()
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backpropagation and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%110==0:
            print (f'Epoch [{epoch+1}/{10}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Testing

In [ ]:
X_test, y_test = read_data(TEST_PATH)

In [ ]:
test_dataset = IntelDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=False
)

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        # predict
        preds = model(images)
        # get the label
        _, predicted = torch.max(preds.data, 1)
        # num of images
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Test Accuracy of the model on the test images: {100 * correct / total}%')